In [16]:
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent
#from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.service import Service
import time
import requests  # For making HTTP requests
from bs4 import BeautifulSoup  # For parsing HTML content
from fake_useragent import UserAgent  # For generating random user agents
import pandas as pd  # For data manipulation and creating DataFrames
import numpy as np
import re
import concurrent.futures
import os

In [18]:
keys = ['LLC','INC','LLP','Corp','LP']

def scrape_all_keys(current):
    service = Service(executable_path=r'C:\Users\H i - G E O R G E\Documents\All_scale\chromedriver.exe')
    options = Options()
    ua = UserAgent()
    userAgent = ua.random
    options.add_argument(f'user-agent={userAgent}')
    driver = webdriver.Chrome(service=service,options=options)
    driver.get('https://business.sos.ri.gov/CorpWeb/CorpSearch/CorpSearch.aspx')
    
    driver.implicitly_wait(5)
    text_field = driver.find_element(By.ID, "MainContent_txtEntityName")
    text_field.clear()
    text_field.send_keys(current)
    time.sleep(2)
    
    search_type_dropdown = driver.find_element(By.ID, "MainContent_ddBeginsWithEntityName")
    select = Select(search_type_dropdown)
    select.select_by_visible_text("Full text")
    time.sleep(2)
    
    dropdown_element = driver.find_element(By.ID, "MainContent_ddRecordsPerPage")
    select1 = Select(dropdown_element)
    select1.select_by_visible_text("100 items")
    
    
    time.sleep(5)
    page = driver.page_source
    
    soup = BeautifulSoup(page, "html.parser")
    soup = soup.find(id="MainContent_SearchControl_grdSearchResultsEntity")
    header = []
    for i in soup.find_all('th'):
        title = i.text.strip() 
        header.append(title)
    
    
    companies = pd.DataFrame(columns=header)
    for j in soup.find_all('tr')[1:]:
        row_data = j.find_all('td')
        if len(row_data) == len(header):
            row = [i.text.strip() for i in row_data]
            companies.loc[len(companies)] = row
    companies = companies.reset_index(drop=True)
    companies['type'] = current
    companies.to_csv(f"companies_{current}.csv")
    print(f'100 items for {current} successfully scraped')
    driver.quit()


for i in keys:
    scrape_all_keys(i)

100 items for LLC successfully scraped
100 items for INC successfully scraped
100 items for LLP successfully scraped
100 items for Corp successfully scraped
100 items for LP successfully scraped


### Merge all csv with name companies in them and deleting after.

In [ ]:
working_dir = r"C:\Users\H i - G E O R G E\Documents\All_scale"
csv_files = [file for file in os.listdir(working_dir) if 'companies' in file and file.endswith('.csv')]

dfs = []
for file in csv_files:
    file_path = os.path.join(working_dir, file)
    df = pd.read_csv(file_path)
    dfs.append(df)


for file in csv_files:
    print(f"Deleting {file}")
    os.remove(os.path.join(working_dir, file))

merged_df = pd.concat(dfs, ignore_index=True)
merged_df.to_csv('companies.csv')

Deleting companies_Corp.csv
Deleting companies_INC.csv
Deleting companies_LLC.csv
Deleting companies_LLP.csv
Deleting companies_LP.csv


In [20]:
import time
import requests  # For making HTTP requests
from bs4 import BeautifulSoup  # For parsing HTML content
from fake_useragent import UserAgent  # For generating random user agents
import pandas as pd  # For data manipulation and creating DataFrames
import numpy as np
import re
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor


companies = pd.read_csv('companies.csv')
companies['ID Number'] = companies['ID Number'].astype(str).str.zfill(9)
   

exact_names = []
fictitious_names = []
entity_types = []
identification_numbers = []
date_of_incorporations = []
effective_dates = []
loc_principal_offices = []
agent_resigned = []
address_maintained = []
name_address_reg_agents = []
off_dir_corps = []
name_business_add_GPs = []
LLC_managers = []
stocks = []
purposes = []
NIACS = []
business_entities = []

# Function to scrape data
def get_more_details(id_num):
    url = f'https://business.sos.ri.gov/CorpWeb/CorpSearch/CorpSummary.aspx?FEIN={id_num}&SEARCH_TYPE=3'
    print(id_num)
    ua = UserAgent()
    userAgent = ua.random
    headers = {'User-Agent': userAgent}
    page = requests.get(url, headers = headers)
    
    if page.status_code != 200:
        return

    soup = BeautifulSoup(page.content, "html.parser")

    def get_text_or_none(soup, tag, attrs):
        element = soup.find(tag, attrs=attrs)
        return element.text if element else 'None'

    exact_names.append(get_text_or_none(soup, 'span', {'id': "MainContent_lblEntityName"}))
    fictitious_names.append(re.sub(r'\s+', ' ', get_text_or_none(soup, 'table', {'id': "MainContent_tblFictitiousName"}).replace('he fictitious name of: ', '')))
    entity_types.append(get_text_or_none(soup, 'span', {'id': "MainContent_lblEntityType"}))
    identification_numbers.append(get_text_or_none(soup, 'span', {'id': "MainContent_lblIDNumber"}).replace('Identification Number: ', ''))
    date_of_incorporations.append(get_text_or_none(soup, 'span', {'id': "MainContent_lblOrganisationDate"}))
    effective_dates.append(get_text_or_none(soup, 'span', {'id': "MainContent_lblEffectiveDate"}))
    loc_principal_offices.append(re.sub(r'\s+', ' ', get_text_or_none(soup, 'table', {'id': "MainContent_tblPrincipleOffice"}).replace('\xa0', '').replace('The location of the Principal Office:', '').replace('Address:', '').replace('City or Town, State, Zip, Country:', ',')))
    agent_resigned.append(get_text_or_none(soup, 'span', {'id': "MainContent_lblResidentAgentFlag"}))
    address_maintained.append(get_text_or_none(soup, 'span', {'id': "MainContent_lblConsentFlag"}))
    name_address_reg_agents.append(re.sub(r'\s+', ' ', get_text_or_none(soup, 'table', {'id': "MainContent_tblResident"}).replace('Name:', '').replace('The name and address of the Registered Agent:', '').replace('Address:', ',').replace('City or Town, State, Zip, Country:', ',').replace('The name and address of the Resident Agent:', '')))
    off_dir_corps.append(re.sub(r'\s+', ' ', get_text_or_none(soup, 'table', {'id': "MainContent_tblOfficers"}).replace('TitleIndividual NameAddress', '').replace('The Officers and Directors of the Corporation:', '').replace('Address:', ',').replace('City or Town, State, Zip, Country:', ',')))
    name_business_add_GPs.append(re.sub(r'\s+', ' ', get_text_or_none(soup, 'table', {'id': "MainContent_grdManagers"}).replace('TitleIndividual nameAddress', '')))
    LLC_managers.append(re.sub(r'\s+', ' ', get_text_or_none(soup, 'table', {'id': "MainContent_tblManagers"}).replace('The limited liability company is to be managed by its ManagersThe name and business address of each Manager:', '').replace('TitleIndividual nameAddress', '')))
    stocks.append(re.sub(r'\s+', ' ', get_text_or_none(soup, 'table', {'id': "MainContent_tblStocks"}).replace('The total number of shares and the par value, if any, of each class of stock which this business entity is authorized to issue:', '').replace('Class of Stock Series Par value per share Total AuthorizedTotal issued and outstandingNo. of sharesNo. of shares', '')))
    purposes.append(re.sub(r'\s+', ' ', get_text_or_none(soup, 'table', {'id': "MainContent_tblComments"}).replace('Purpose:', '')))
    NIACS.append(get_text_or_none(soup, 'input', {'id': "MainContent_txtNIACS", "value": True}))
    business_entities.append(re.sub(r'\s+', ' ', get_text_or_none(soup, 'tr', {'id': "MainContent_tdFilingList"}).replace('\n', ',').strip(',')))


# Use ThreadPoolExecutor to scrape in parallel
with ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(get_more_details, companies['ID Number'])

# Create DataFrame
extra_comp_data = {
    "Exact Names": exact_names,
    "Fictitious Names and Filed Date": fictitious_names,
    "Entity Types": entity_types,
    "Identification Numbers": identification_numbers,
    "Date of Incorporations": date_of_incorporations,
    "Effective Dates": effective_dates,
    "Location of Principal Offices": loc_principal_offices,
    "Agent Resigned": agent_resigned,
    "Address Maintained": address_maintained,
    "Name and Address of Registered Agents": name_address_reg_agents,
    "Officers, Directors, and Corporations": off_dir_corps,
    "Name and Business Address of General Partners": name_business_add_GPs,
    "LLC Managers": LLC_managers,
    "Stocks": stocks,
    "Purposes": purposes,
    "NIACS": NIACS,
    "Business Entities": business_entities
}

full_comp_df = pd.DataFrame(extra_comp_data)
full_comp_df.to_csv('full_details.csv')


001761799000089282

000151557
000005981
000034847
001779503
000076285
000076283
000115782
000115783
000065408
000487970
001731359
000791798
000075075
001698885
001737982
000075278
001690061
000010227
000789962
000088725
001761540
000906409
001670100
001752136
000163473
000151555
000001271
000087492
001765705
000522037
001688187
001767573
000830351
000118376
000118376
001723685
000095225
000118376
000118376
001763258
000035416
000090193
000090981
001685268
000557420
000000241
001683530
001767754
000293567
000068312
000682435
001700208
001730918
001730909
000883001
000567302
001772649
001775633
000506278
001773989
000516372
001685835
000923522
001767988
000045546
000090193
000131487
000120298
001780406
000443186
000088176
000045508000045508

000045508
000061591
001761425
001682453
001751437
001762367
000157003
000114469
000487606
001782121
001782121
000000558
000304843
001750123
000129830000041942

001718605
000099804
000100751
000069056
001769757
000546599
001757014
000113902
000571487
